In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

#year = 2022
name = 'STA'
buy_date = '2021-06-15'

In [2]:
format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
              'pct':'{:,.2f}%','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for transactions

In [3]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND buys.date >= '%s'
ORDER BY sell_year DESC, sell_month DESC, name'''

sql = sql % (name, buy_date)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = 'STA' AND buys.date >= '2021-06-15'
ORDER BY sell_year DESC, sell_month DESC, name


In [4]:
sells_df

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,profit,kind


### Record selection for active stocks

In [7]:
sql = '''
SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND status = 'Active'
ORDER BY name, buys.date'''
sql = sql % name
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,buy_date,unit_cost,qty,cost_amt,net
0,STA,2021-06-15,40.00,"2,500","100,000.00","100,221.49"
1,STA,2021-06-21,40.00,"2,500","100,000.00","100,221.49"
2,STA,2021-07-21,37.75,"2,500","94,375.00","94,584.04"
3,STA,2021-09-13,35.25,"2,500","88,125.00","88,320.19"
4,STA,2021-09-23,33.25,"2,500","83,125.00","83,309.11"
5,STA,2021-10-27,32.75,"2,500","81,875.00","82,056.34"
6,STA,2021-12-13,29.50,"2,500","73,750.00","73,913.35"
7,STA,2021-12-20,29.50,"2,500","73,750.00","73,913.35"


In [8]:
buys_df[['cost_amt','net','qty']].sum()

cost_amt   695,000.00
net        696,539.36
qty         20,000.00
dtype: float64

In [9]:
sql = '''
SELECT name, p_date, amt, net, amt-net AS refund,
year, quarter AS q, x_date, number, ppu, (net / number * 1000) AS div_1k
FROM dividends
WHERE name = '%s' AND x_date > '%s'
ORDER BY p_date DESC'''
sql = sql % (name, buy_date)
div_df = pd.read_sql(sql, conpf)
div_df

,name,p_date,amt,net,refund,year,q,x_date,number,ppu,div_1k
0,STA,2021-12-08,"18,750.00","16,875.00","1,875.00",2021,3,2021-11-19,15000,1.25,"1,125.00"
1,STA,2021-09-08,"9,375.00","8,625.00",750.00,2021,2,2021-08-23,7500,1.25,"1,150.00"


In [10]:
div_grp = div_df.groupby(['name'], as_index=True).agg(
    {
        'amt':['sum'],
        'net':['sum'], 
        'number':['mean','count'],         
    }
)
div_grp

amt       net    number      
           sum       sum      mean count
name                                    
STA  28,125.00 25,500.00 11,250.00     2

In [11]:
acc_div = 25500
acc_div

25500

In [24]:
sql = '''
SELECT name, buys.date AS buy_date,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt, net
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE name = '%s' AND status = 'Active'
AND buys.date > '2021-06-21'
ORDER BY name, buys.date'''
sql = sql % name
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,buy_date,unit_cost,qty,cost_amt,net
0,STA,2021-07-21,37.75,"2,500","94,375.00","94,584.04"
1,STA,2021-09-13,35.25,"2,500","88,125.00","88,320.19"
2,STA,2021-09-23,33.25,"2,500","83,125.00","83,309.11"
3,STA,2021-10-27,32.75,"2,500","81,875.00","82,056.34"
4,STA,2021-12-13,29.50,"2,500","73,750.00","73,913.35"
5,STA,2021-12-20,29.50,"2,500","73,750.00","73,913.35"


In [25]:
buys_df[['qty','cost_amt']].sum()

qty         15,000.00
cost_amt   495,000.00
dtype: float64

In [27]:
sell_price = 31.5
sales_amt = sell_price * 15_000
offset_amt = sales_amt + acc_div
offset_amt

498000.0

### Plan A: Sell at 31.50 x 15,000 shares